In [ ]:
0.5045877 194 место

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv('/media/derin/DATA/Machine Learning/Megafon/condition_csi/dataset/train/subs_features_train.csv',sep=";")

In [5]:
test = pd.read_csv('/media/derin/DATA/Machine Learning/Megafon/condition_csi/dataset/test/subs_features_test.csv',sep=";")

In [6]:
train = train.sort_values('SK_ID', ascending=False)

In [7]:
test = test.sort_values('SK_ID', ascending=False)

In [8]:
train = train.reset_index(drop = True) # делаю индексы 0,1,2...

In [9]:
test = test.reset_index(drop = True)  # делаю индексы 0,1,2...

In [10]:
#результаты для train
ans = pd.read_csv('/media/derin/DATA/Machine Learning/Megafon/condition_csi/dataset/train/subs_csi_train.csv',sep=";")

In [11]:
train = train.replace(np.nan, -1)

In [12]:
test = test.replace(np.nan, -1)

In [13]:
#убираем SNAP_DATE
train = train[['COM_CAT#1', 'SK_ID', 'COM_CAT#2', 'COM_CAT#3',        'BASE_TYPE', 'ACT', 'ARPU_GROUP', 'COM_CAT#7', 'COM_CAT#8', 'DEVICE_TYPE_ID', 'INTERNET_TYPE_ID', 'REVENUE', 'ITC', 'VAS', 'RENT_CHANNEL', 'ROAM', 'COST', 'COM_CAT#17', 'COM_CAT#18', 'COM_CAT#19', 'COM_CAT#20', 'COM_CAT#21', 'COM_CAT#22', 'COM_CAT#23', 'COM_CAT#24', 'COM_CAT#25', 'COM_CAT#26', 'COM_CAT#27', 'COM_CAT#28', 'COM_CAT#29', 'COM_CAT#30', 'COM_CAT#31', 'COM_CAT#32', 'COM_CAT#33', 'COM_CAT#34']]
test = test[['COM_CAT#1', 'SK_ID', 'COM_CAT#2', 'COM_CAT#3',        'BASE_TYPE', 'ACT', 'ARPU_GROUP', 'COM_CAT#7', 'COM_CAT#8', 'DEVICE_TYPE_ID', 'INTERNET_TYPE_ID', 'REVENUE', 'ITC', 'VAS', 'RENT_CHANNEL', 'ROAM', 'COST', 'COM_CAT#17', 'COM_CAT#18', 'COM_CAT#19', 'COM_CAT#20', 'COM_CAT#21', 'COM_CAT#22', 'COM_CAT#23', 'COM_CAT#24', 'COM_CAT#25', 'COM_CAT#26', 'COM_CAT#27', 'COM_CAT#28', 'COM_CAT#29', 'COM_CAT#30', 'COM_CAT#31', 'COM_CAT#32', 'COM_CAT#33', 'COM_CAT#34']] 

In [14]:
numbers_features = []

In [15]:
#преобразуем str в float64 для train,03434 -> 0.03434
for name in train.columns:
    if (type(train[name][0]) == type('')):
        train[name] = np.float64('0' + train[name].str.replace(',', '.'))
        numbers_features.append(name)

In [16]:
#преобразуем str в float64 для test,03434 -> 0.03434
for name in test.columns:
    if (type(test[name][0]) == type('')):
        test[name] = np.float64('0' + test[name].str.replace(',', '.'))

In [17]:
#количественные признаки
numbers_features

['REVENUE',
 'ITC',
 'VAS',
 'RENT_CHANNEL',
 'ROAM',
 'COST',
 'COM_CAT#17',
 'COM_CAT#18',
 'COM_CAT#19',
 'COM_CAT#20',
 'COM_CAT#21',
 'COM_CAT#22',
 'COM_CAT#23',
 'COM_CAT#27',
 'COM_CAT#28',
 'COM_CAT#29',
 'COM_CAT#30',
 'COM_CAT#31',
 'COM_CAT#32',
 'COM_CAT#33']

In [18]:
#качественные признаки
type_features = []
for i in train.columns.values:
    if not i in numbers_features:
        type_features.append(i)

In [21]:
#преобразуем train/test для одного SK_ID - одна строка (значения для качественных признаков - голосованием,
#для количественных - медианой) 
def new(df):
    new_df = pd.DataFrame()
    for name in train.columns.values:
        new_df[name] = [np.float32(-1.0)]
    new_df['SK_ID'] = [int(-1)]
    for cur_id in df['SK_ID'].unique():
        part_df = df[df['SK_ID'] == cur_id]
        new_line = copy.copy(new_df.loc[0])
        for name in new_df.columns.values:
                if name in type_features:
                    new_line[name] = part_df[name].value_counts().idxmax()
                else:
                    new_line[name] = part_df[name].median()
                            
        new_line['SK_ID'] = cur_id
        new_df.loc[np.shape(new_df)[0]] = new_line     
        
        
    return new_df

In [22]:
ntrain = new(train)

In [23]:
ntest = new(test)

In [24]:
#убираем первую строку(она была техническая, чтобы dataframe сделать)
ntrain = ntrain.loc[1:]

In [36]:
ntrain = ntrain.reset_index(drop = True) #делаем индексы 0,1,2...

In [25]:
#убираем первую строку(она была техническая, чтобы dataframe сделать)
ntest = ntest.loc[1:]

In [38]:
ntest = ntest.reset_index(drop = True) #делаем индексы 0,1,2...

In [26]:
#делаем в ntrain столбы такого же типа, как в train(они почему-то были все float)
for name in ntrain.columns.values:
    ntrain[name] = type(train[name].loc[3])(ntrain[name])

In [27]:
#делаем в ntrain столбы такого же типа, как в train(они почему-то были все float)
for name in ntest.columns.values:
    ntest[name] = type(test[name].loc[3])(ntest[name])

In [28]:
# ans - это subs_csi_train - то есть 0 и 1 в train
ans = ans.sort_values('SK_ID', ascending=False)

In [42]:
ans = ans.reset_index(drop = True) #индексы 0,1,2...

In [29]:
#это те индексы, для которых мы предсказываем
index_test = pd.read_csv('/media/derin/DATA/Machine Learning/Megafon/condition_csi/dataset/test/subs_csi_test.csv',sep=";")

In [53]:
#так формирую DataFrame
itog_test = ntest.loc[0:0]

In [54]:
itog_test

,COM_CAT#1,SK_ID,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,COM_CAT#7,COM_CAT#8,DEVICE_TYPE_ID,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,1,9748,30,5,2,1,6.0,7,1325.0,5.0,...,1,1,0.001855,0.004912,0.000064,0.000034,0.006705,0.027579,0.103896,9.0


In [55]:
#делаю для id из index_test - строки из subs_features_test.csv(создаю test, 
#в котором SK_ID в том же порядке, как требуют)
lens_itog = 1
for cur_id in index_test['SK_ID']:
    if (np.shape(ntest[ntest['SK_ID'] == cur_id])[0] > 0):
        itog_test.loc[lens_itog] = (ntest[ntest['SK_ID'] == cur_id].reset_index()).loc[0]
    else:
        nl = ntrain.loc[:1]
        for name1 in nl.columns.values:
            nl[name1] = -1
        nl['SK_ID'] = cur_id
        itog_test.loc[lens_itog] = nl.loc[1]
    lens_itog += 1
    
    

/home/derin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [57]:
itog_test.head()

,COM_CAT#1,SK_ID,COM_CAT#2,COM_CAT#3,BASE_TYPE,ACT,ARPU_GROUP,COM_CAT#7,COM_CAT#8,DEVICE_TYPE_ID,...,COM_CAT#25,COM_CAT#26,COM_CAT#27,COM_CAT#28,COM_CAT#29,COM_CAT#30,COM_CAT#31,COM_CAT#32,COM_CAT#33,COM_CAT#34
0,1.0,9748.0,30.0,5.0,2.0,1.0,6.0,7.0,1325.0,5.0,...,1.0,1.0,0.001855,0.004912,0.000064,0.000034,0.006705,0.027579,0.103896,9.0
1,4.0,6184.0,50.0,5.0,2.0,1.0,6.0,2.0,136.0,5.0,...,1.0,1.0,0.009586,0.010298,0.000044,0.000188,0.019549,0.027579,0.103896,9.0
2,3.0,5175.0,24.0,5.0,2.0,1.0,10.0,1.0,149.0,5.0,...,1.0,1.0,0.034592,0.030623,0.063411,0.000237,0.048608,0.027579,0.103896,2.0
3,6.0,5478.0,55.0,5.0,2.0,1.0,5.0,3.0,962.0,3.0,...,1.0,1.0,0.015872,0.000800,0.000000,0.000000,0.090086,0.027579,0.103896,1.0
4,6.0,7807.0,71.0,9.0,1.0,1.0,6.0,2.0,144.0,5.0,...,1.0,1.0,0.008567,0.014240,0.010308,0.000053,0.065341,0.027579,0.103896,-1.0


In [58]:
#убираю строку(которой формировала DataFrame)
itog_test = ((itog_test.reset_index(drop = True)).loc[1:]).reset_index(drop = True)

In [59]:
np.shape(itog_test)

(5221, 35)

In [60]:
y = ans['CSI']

In [63]:
X = ntrain.values

In [65]:
np.shape(X)

(4222, 35)

In [66]:
import sklearn.ensemble
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [67]:
calc(sklearn.ensemble.RandomForestClassifier(), X, y)

/home/derin/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/derin/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/derin/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/derin/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/derin/.local/lib/python3.5/site-packages/sklearn/ensem

0.5047759613528944

In [68]:
Xt = itog_test.values

In [69]:
y = y.values

In [70]:
import sklearn.ensemble
model = sklearn.ensemble.RandomForestClassifier()
model.fit(X, y)

/home/derin/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
y_pred = model.predict(Xt)

In [72]:
len(y_pred)

5221

In [79]:
for i in range(len(y_pred)):
    if (y_pred[i] == 0):
        y_pred[i] = 1
    else:
        y_pred[i] = 0


In [80]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [81]:
np.shape(y_pred)

(5221,)

In [82]:
pd.DataFrame({'a': y_pred}).to_csv('sub_second_2.csv')